In [1]:
import pandas as pd
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from tqdm import tqdm
import numpy as np
import scann

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2024-02-26 13:27:13.187883: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 13:27:14.392072: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 13:27:14.396731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-26 13:27:15.944656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [31]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/home/ubuntu/infoedge/llama-recipes/env/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [8]:
df =pd.read_csv('../../examples/relevant_jobs_classification/data/analysis_dataset.csv')

In [9]:
df

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [10]:
df.job_id.nunique()

2949

In [11]:
df.user_id.nunique()

30363

In [12]:
import requests

def fetch_jobs(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    

    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    # print(response.text)
    

    # url = url.replace("<DAYS>",str(last_k)).replace("<SIZE>",str(size))

    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()
    # print(job_data)
    job_df = pd.DataFrame(job_data)

    return job_df[['id','introText']]

In [13]:

jd = fetch_jobs('1373869')

In [14]:
jd

,id,introText
0,1373869,<p>Sr. Brand Marketing Manager </p><p><br/></p...


In [15]:
# df2 = df.head(100)

In [16]:
# df2

In [17]:
df.job_id.nunique()

2949

In [18]:
df.user_id.nunique()

30363

In [19]:
from tqdm import tqdm
df_list = []
for i in tqdm(df['job_id'].unique()):
    try:
        j = fetch_jobs(str(i))
    
        df_list.append(j)
    except:
        continue
jd = pd.concat(df_list)    

    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2949/2949 [00:39<00:00, 74.79it/s]


In [20]:
jd

,id,introText
0,1373681,<p>About Company - Global recognized leader in...
0,1373682,<p>About Company - Global recognized leader in...
0,1373683,<p>A prominent global investment and technolog...
0,1373684,<p>Limese is looking for a detail-oriented and...
0,1373685,<p>The RM shall be responsible for originating...
...,...,...
0,1376446,<p>Digital Analytics - Domain Specialist <br/>...
0,1376758,<p>Urban Company is a tech-enabled home servic...
0,1375146,<p>Reinsurance Manager - General Insurance </p...
0,1376759,<p>Experience in managing Architects Developer...


In [21]:
# 2949-2824

In [22]:
# d.columns = ['job_id','introText']

In [23]:
jid_list = list(jd.id)

In [24]:
# df

In [25]:
df1 = df.loc[df['job_id'].isin(jid_list)]

In [26]:
jd.columns = ['job_id','introText']

In [27]:
jd

,job_id,introText
0,1373681,<p>About Company - Global recognized leader in...
0,1373682,<p>About Company - Global recognized leader in...
0,1373683,<p>A prominent global investment and technolog...
0,1373684,<p>Limese is looking for a detail-oriented and...
0,1373685,<p>The RM shall be responsible for originating...
...,...,...
0,1376446,<p>Digital Analytics - Domain Specialist <br/>...
0,1376758,<p>Urban Company is a tech-enabled home servic...
0,1375146,<p>Reinsurance Manager - General Insurance </p...
0,1376759,<p>Experience in managing Architects Developer...


In [28]:
merged_df = pd.merge(jd, df1, on='job_id', how='left')


In [29]:
merged_df.sample(5)

,job_id,introText,application_id,user_id
81951,1375583,<p>Functional Capability:<br/><br/>- Contribut...,144316769,2556593
5037,1373781,<p>Director - Business Operations </p><p><br/>...,144168052,524961
81999,1375583,<p>Functional Capability:<br/><br/>- Contribut...,144414729,660695
17561,1373949,<p><b>About FreeUp:</b><br/><br/>FreeUp is Ind...,144167254,1105121
95929,1375991,<p>Designation - Zonal Operations Manager<br/>...,144414828,2792349


In [30]:
merged_df

,job_id,introText,application_id,user_id
0,1373681,<p>About Company - Global recognized leader in...,144274567,439042
1,1373681,<p>About Company - Global recognized leader in...,144261479,2015063
2,1373681,<p>About Company - Global recognized leader in...,144253966,2777927
3,1373681,<p>About Company - Global recognized leader in...,144241259,58130
4,1373681,<p>About Company - Global recognized leader in...,144236812,2290933
...,...,...,...,...
109521,1376758,<p>Urban Company is a tech-enabled home servic...,144442159,1311734
109522,1375146,<p>Reinsurance Manager - General Insurance </p...,144438242,1824334
109523,1376759,<p>Experience in managing Architects Developer...,144442110,1619508
109524,1376737,<p>Executive Hiring is looking for a Performan...,144442121,1352791


In [32]:
uid_list = list(merged_df.user_id.unique())

In [33]:
def clean_space(text):
    return " ".join(re.split("\s+", text, flags=re.UNICODE))

def remove_extra_spaces(input_string):
    # Use regular expression to remove extra spaces (only spaces)
    cleaned_string = re.sub(r' +', ' ', input_string)
    return cleaned_string
     

def get_user_data_search_embed(user_id_list):
    
    '''
    This function combines all the functions for fetching and processing 
    user data from elastic search
    '''


    sub_json = fetch_data_es(user_id_list)
    raw_data = construct_user_data_search_embed(sub_json)

    res_df = pd.DataFrame.from_dict(raw_data, orient='index').T
    result_df = res_df.astype(str).replace("None", " ")
    
    result_df['resume'] = result_df['resume'].apply(remove_extra_spaces)

    return result_df

def fetch_data_es(user_id_list):
    
    """
    Input : List of Valid Hirist User ID 
    Output : List of User Data JSON stored in Elastic Search
    """
    
    
    response_json = []
    replacements = {'null':'None', 'false':'False'}

    url = "http://10.208.230.226:8080/v1/recruiter/-123/applicant/search/searchById"

    res = []

    for uid in tqdm(user_id_list):


        payload = {
            'usersToSearchFrom' : [str(uid)]
        }

        headers = {
        'Content-Type': 'application/json'
        }

        try:
            response = requests.request("POST", url, headers=headers, data = str(payload))
            response_json = response.json()
    
            res.append(response_json['docs']) 

        except Exception as e:
            continue
        
    return res



def construct_user_data_search_embed(user_json):
    '''
    This is a function for fetching user ID, Keywords (in the form of tags) and 
    user designation from elastic search. This is specifically made for creating 
    user embeddings for our Recruiter search model/POC
    '''
    
    user_dict = {
        'id' : [],
        'resume' : []
    }

    for s in tqdm(user_json):

        try:
            source = s[0]

            user_dict['id'].append(source['id'])
            user_dict['resume'].append(clean_space(source['resumeText']))

        except Exception as e:
            continue

    return user_dict

In [34]:

def get_user_data_in_batches(uid_list, batch_size=1000):
    user_data = pd.DataFrame()

    for i in range(0, len(uid_list), batch_size):
        batch_uids = uid_list[i:i + batch_size]

        batch_user_data = get_user_data_search_embed(batch_uids)

        user_data = pd.concat([user_data, batch_user_data], ignore_index=True)

    return user_data

In [35]:
batched_user_data = get_user_data_in_batches(uid_list, batch_size=1000)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 885/885 [00:00<00:00, 4552.51it/s]


In [36]:
len(uid_list)

26885

In [37]:
batched_user_data.columns = ['user_id','resume']

In [38]:
batched_user_data.dtypes

user_id    object
resume     object
dtype: object

In [39]:
batched_user_data['user_id']=batched_user_data['user_id'].astype(int)

In [40]:
main_df=pd.merge(merged_df,batched_user_data, on='user_id', how='left')

In [41]:
main_df.head()

,job_id,introText,application_id,user_id,resume
0,1373681,<p>About Company - Global recognized leader in...,144274567,439042,Vibha Issar Email: vibhaissar@rediffmail.com M...
1,1373681,<p>About Company - Global recognized leader in...,144261479,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...
2,1373681,<p>About Company - Global recognized leader in...,144253966,2777927,Head Office Business process review and revamp...
3,1373681,<p>About Company - Global recognized leader in...,144241259,58130,WORK EXPERIENCE SUMMARY &amp; KEY ACHIEVEMENTS...
4,1373681,<p>About Company - Global recognized leader in...,144236812,2290933,WORK EXPERIENCE (5Y 5M) YES BANK | APRIL 2022 ...


In [42]:
main_df.isnull().sum()

job_id              0
introText           0
application_id      0
user_id             0
resume            973
dtype: int64

In [43]:
# main_df.loc[main_df['job_id']==1373681]

In [44]:
def clean_text(input_text):
    
    '''Make text lowercase, remove text in square brackets,remove links,
    remove punctuation and remove words containing numbers.'''

    # This code block was added to remove the company names from Job embeddings
    
    # t = input_text.split('-')
    # if len(t)>3:
    #     t1 = t[1:]
    #     text = ' '.join(t1)
    # else:
    #     text = input_text
    #     pass

    text = input_text
        
    text = str(text).lower()
    text = re.sub('\[.*?\]',' ', text)
    text = re.sub('https?://\S+|www\.\S+',' ', text)
    text = re.sub('<.*?>+',' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation),' ', text)
    text = re.sub('\n',' ', text)
    text = re.sub('\w*\d\w*',' ', text)
    text = re.sub('yrs',' ',text)
    text = re.sub('\r',' ',text)
    text = ' '.join([word for word in text.split() if word not in (stop)])
    text = re.sub(' +', ' ', text)
    
    return text

In [45]:
li = []
for d in tqdm(main_df['introText']):
    des = clean_text(d)
    li.append(des)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109526/109526 [01:20<00:00, 1361.48it/s]


In [47]:
main_df['description']=li

In [48]:
main_df

,job_id,introText,application_id,user_id,resume,description
0,1373681,<p>About Company - Global recognized leader in...,144274567,439042,Vibha Issar Email: vibhaissar@rediffmail.com M...,company global recognized leader financial tec...
1,1373681,<p>About Company - Global recognized leader in...,144261479,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...,company global recognized leader financial tec...
2,1373681,<p>About Company - Global recognized leader in...,144253966,2777927,Head Office Business process review and revamp...,company global recognized leader financial tec...
3,1373681,<p>About Company - Global recognized leader in...,144241259,58130,WORK EXPERIENCE SUMMARY &amp; KEY ACHIEVEMENTS...,company global recognized leader financial tec...
4,1373681,<p>About Company - Global recognized leader in...,144236812,2290933,WORK EXPERIENCE (5Y 5M) YES BANK | APRIL 2022 ...,company global recognized leader financial tec...
...,...,...,...,...,...,...
109521,1376758,<p>Urban Company is a tech-enabled home servic...,144442159,1311734,Profile Summary | Direct Sales Distribution| S...,urban company tech enabled home services marke...
109522,1375146,<p>Reinsurance Manager - General Insurance </p...,144438242,1824334,"JINIMOL SKARIA (CA, B. Com, LIII) Objective: T...",reinsurance manager general insurance location...
109523,1376759,<p>Experience in managing Architects Developer...,144442110,1619508,Gaurav Rajendra Nemade 08/12/1994|nemadegaurav...,experience managing architects developers role...
109524,1376737,<p>Executive Hiring is looking for a Performan...,144442121,1352791,Yashodhan Joshi Phonenumber: 8009901122 Emaila...,executive hiring looking performance marketing...


In [49]:
# main_df[['resume']].values

In [50]:
# main_df.to_csv('../custom_data/metric_data/main_data.csv', index=False)

## Start here

In [51]:
# import tensorflow_hub as hub
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [52]:
# df = pd.read_csv('../custom_data/metric_data/main_data.csv')

In [53]:
# main_df

In [54]:
li = []
for r in tqdm(main_df['resume']):
    res = clean_text(r)
    li.append(res)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109526/109526 [05:27<00:00, 334.67it/s]


In [55]:
main_df['resume_text']=li

In [56]:
# df[['resume','resume_text']][:2].values

In [57]:
main_df

,job_id,introText,application_id,user_id,resume,description,resume_text
0,1373681,<p>About Company - Global recognized leader in...,144274567,439042,Vibha Issar Email: vibhaissar@rediffmail.com M...,company global recognized leader financial tec...,vibha issar email vibhaissar rediffmail com mo...
1,1373681,<p>About Company - Global recognized leader in...,144261479,2015063,Swati Kamble +91 913 7083 435 | swatikamble257...,company global recognized leader financial tec...,swati kamble gmail com expert financial servic...
2,1373681,<p>About Company - Global recognized leader in...,144253966,2777927,Head Office Business process review and revamp...,company global recognized leader financial tec...,head office business process review revamping ...
3,1373681,<p>About Company - Global recognized leader in...,144241259,58130,WORK EXPERIENCE SUMMARY &amp; KEY ACHIEVEMENTS...,company global recognized leader financial tec...,work experience summary amp key achievements t...
4,1373681,<p>About Company - Global recognized leader in...,144236812,2290933,WORK EXPERIENCE (5Y 5M) YES BANK | APRIL 2022 ...,company global recognized leader financial tec...,work experience yes bank april march stressed ...
...,...,...,...,...,...,...,...
109521,1376758,<p>Urban Company is a tech-enabled home servic...,144442159,1311734,Profile Summary | Direct Sales Distribution| S...,urban company tech enabled home services marke...,profile summary direct sales distribution stra...
109522,1375146,<p>Reinsurance Manager - General Insurance </p...,144438242,1824334,"JINIMOL SKARIA (CA, B. Com, LIII) Objective: T...",reinsurance manager general insurance location...,jinimol skaria ca b com liii objective advance...
109523,1376759,<p>Experience in managing Architects Developer...,144442110,1619508,Gaurav Rajendra Nemade 08/12/1994|nemadegaurav...,experience managing architects developers role...,gaurav rajendra nemade nemadegaurav gmail com ...
109524,1376737,<p>Executive Hiring is looking for a Performan...,144442121,1352791,Yashodhan Joshi Phonenumber: 8009901122 Emaila...,executive hiring looking performance marketing...,yashodhan joshi phonenumber emailaddress gmail...


In [58]:
mdf = main_df[['job_id','description','user_id','resume_text']]

In [59]:
mdf

,job_id,description,user_id,resume_text
0,1373681,company global recognized leader financial tec...,439042,vibha issar email vibhaissar rediffmail com mo...
1,1373681,company global recognized leader financial tec...,2015063,swati kamble gmail com expert financial servic...
2,1373681,company global recognized leader financial tec...,2777927,head office business process review revamping ...
3,1373681,company global recognized leader financial tec...,58130,work experience summary amp key achievements t...
4,1373681,company global recognized leader financial tec...,2290933,work experience yes bank april march stressed ...
...,...,...,...,...
109521,1376758,urban company tech enabled home services marke...,1311734,profile summary direct sales distribution stra...
109522,1375146,reinsurance manager general insurance location...,1824334,jinimol skaria ca b com liii objective advance...
109523,1376759,experience managing architects developers role...,1619508,gaurav rajendra nemade nemadegaurav gmail com ...
109524,1376737,executive hiring looking performance marketing...,1352791,yashodhan joshi phonenumber emailaddress gmail...


In [60]:
mdf.dtypes

job_id          int64
description    object
user_id         int64
resume_text    object
dtype: object

In [61]:
# mdf.to_csv('../custom_data/metric_data/main_df.csv',index=False)

In [62]:
# j = mdf.job_id.sample().values[0]

In [63]:
# j

In [64]:

# df_list = []

# for jid in tqdm(mdf.job_id.unique()):
#     # print(jid)
#     jobdf = mdf.loc[mdf['job_id']==jid]

#     description = jobdf.description.sample().values[0]

#     job_embeddings = embed([description]).numpy()[0]
    

#     udf = jobdf[['user_id', 'resume_text']]
    
#     udf_list = udf.values
    
#     r_list = udf_list[:,1].tolist()
#     uid_list = udf_list[:,0].tolist()
    
#     try:
    
#         embedding_sublist = [e for e in embed(r_list).numpy()]
#     except Exception as e:
#         continue

#     sub_dict = {uid: np.array([embedding]) for uid, embedding in zip(uid_list, embedding_sublist)}
    
#     # start = time.time()

#     uv = embedding_sublist

#     try:
#         nu = uv / np.linalg.norm(uv, axis=1)[:, np.newaxis]
#     except Exception as e:
#         print(f'Error is {e}')  
#         continue
    
#     searcher = scann.scann_ops_pybind.builder(nu, 10000, "dot_product").score_brute_force(2).build()

    
#     neighbours, distances = searcher.search_batched([job_embeddings])
    
#     udf = udf.iloc[neighbours[0]]
#     udf['scores'] = distances[0] 
#     udf.drop(['resume_text'],axis=1,inplace=True)

#     final_df=pd.merge(jobdf,udf, on='user_id', how='left')

#     df_list.append(final_df)
    
# result_df =pd.concat(df_list)
#     # m1.loc[m1['job_id'] == jid, 'scores'] = distances[0]

    


## Main code

In [65]:
# mdf = pd.read_csv('../custom_data/metric_data/main_df.csv')

In [66]:
# mdf

In [67]:
# embed([mdf.description.sample().values[0]]).numpy()[0]

In [68]:
# mdf.loc[mdf['user_id']==2459415]['resume_text'].values[0]

In [69]:
print('Start from here')

Start from here


In [70]:
mdf['job_id'].value_counts().max()

815

In [71]:
# mdf.loc[mdf['user_id']==439042]['resume_text'].values[0]

In [73]:
import numpy as np
from tqdm import tqdm


batch_size = 10  # Adjust the batch size according to your system's capacity

df_list = []

unique_job_ids = mdf.job_id.unique()

num_batches = len(unique_job_ids) // batch_size + 1

for batch_num in tqdm(range(num_batches)):
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size
    batch_job_ids = unique_job_ids[start_idx:end_idx]

    batch_df_list = []

    try:

        for jid in batch_job_ids:
            jobdf = mdf.loc[mdf['job_id'] == jid]
    
            description = jobdf.description.sample().values[0]
            job_embeddings = model.encode([description])[0]
    
            udf = jobdf[['user_id', 'resume_text']]
            udf_list = udf.values
            r_list = udf_list[:, 1].tolist()
            uid_list = udf_list[:, 0].tolist()
            
            try:
                embedding_sublist = [e for e in model.encode(r_list)]
            except Exception as e:
                # print(uid_list)
                print('problem')
                continue
    
            sub_dict = {uid: np.array([embedding]) for uid, embedding in zip(uid_list, embedding_sublist)}
    
            uv = embedding_sublist
    
            try:
                nu = uv / np.linalg.norm(uv, axis=1)[:, np.newaxis]
            except Exception as e:
                print(f'Error is {e}')
    
            searcher = scann.scann_ops_pybind.builder(nu, 900, "dot_product").score_brute_force(2).build()

        #     searcher = scann.scann_ops_pybind.builder(nu, 900, "dot_product").tree(
        # num_leaves=10, num_leaves_to_search=10, training_sample_size=1000).score_ah(
        # 2, anisotropic_quantization_threshold=0.2).reorder(1000).build()

    
            neighbours, distances = searcher.search_batched([job_embeddings])
    
            udf = udf.iloc[neighbours[0]]
            udf['scores'] = distances[0]
            udf.drop(['resume_text'], axis=1, inplace=True)
    
            final_df = pd.merge(jobdf, udf, on='user_id', how='left')
            batch_df_list.append(final_df)            
            
            del jobdf, udf, embedding_sublist, nu, searcher

    
        batch_result_df = pd.concat(batch_df_list)
        df_list.append(batch_result_df)
    except Exception as e:
        print(e)
        continue
result_df = pd.concat(df_list)


  0%|                                                                                                                                              | 0/233 [00:00<?, ?it/s][libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannConfig: 7:20: Integer out of range (2)
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format research_scann.ScannCon

In [74]:
result_df

,job_id,description,user_id,resume_text,scores
0,1373681,company global recognized leader financial tec...,439042,vibha issar email vibhaissar rediffmail com mo...,0.709908
1,1373681,company global recognized leader financial tec...,2015063,swati kamble gmail com expert financial servic...,0.712206
2,1373681,company global recognized leader financial tec...,2777927,head office business process review revamping ...,0.853119
3,1373681,company global recognized leader financial tec...,58130,work experience summary amp key achievements t...,0.806288
4,1373681,company global recognized leader financial tec...,2290933,work experience yes bank april march stressed ...,0.763922
...,...,...,...,...,...
15,1376758,urban company tech enabled home services marke...,1311734,profile summary direct sales distribution stra...,0.623250
0,1375146,reinsurance manager general insurance location...,1824334,jinimol skaria ca b com liii objective advance...,0.780057
0,1376759,experience managing architects developers role...,1619508,gaurav rajendra nemade nemadegaurav gmail com ...,0.662120
0,1376737,executive hiring looking performance marketing...,1352791,yashodhan joshi phonenumber emailaddress gmail...,0.731741


In [78]:
result_df.loc[result_df['job_id']==1373684].head(50)

,job_id,description,user_id,resume_text,scores
0,1373684,limese looking detail oriented analytically sk...,2280817,october current may october pranav jaiswal sup...,0.834983
1,1373684,limese looking detail oriented analytically sk...,2730065,prakash murugesan india mobile email gmail com...,0.789003
2,1373684,limese looking detail oriented analytically sk...,2354895,rahul singh email rahulsingh gmail com linkedi...,0.677909
3,1373684,limese looking detail oriented analytically sk...,2711440,akhil pratap singh asian paints ltd central su...,0.714604
4,1373684,limese looking detail oriented analytically sk...,2500298,piyush kumar patel gmail com mumbai maharashtr...,0.724914
5,1373684,limese looking detail oriented analytically sk...,1229742,work experience projects internships abhishek ...,0.711241
6,1373684,limese looking detail oriented analytically sk...,2681866,vignesh iyer new delhi gmail com linkedin com ...,0.752077
7,1373684,limese looking detail oriented analytically sk...,2436531,amit ahirwar male dob nov gmail com scholastic...,0.753868
8,1373684,limese looking detail oriented analytically sk...,2806722,mithila kirve mumbai maharashtra india mobile ...,0.588889
9,1373684,limese looking detail oriented analytically sk...,218127,rahul boriwal email id gmail com contact profe...,0.603976


In [77]:
# result_df.to_csv('../../examples/relevant_jobs_classification/data/score_result.csv',index=False)

In [77]:
# udf

In [81]:
result_df.loc[result_df['job_id']==1373684].head(50)

,job_id,description,user_id,resume_text,scores
0,1373684,limese looking detail oriented analytically sk...,2280817,october current may october pranav jaiswal sup...,0.729711
1,1373684,limese looking detail oriented analytically sk...,2730065,prakash murugesan india mobile email gmail com...,0.627667
2,1373684,limese looking detail oriented analytically sk...,2354895,rahul singh email rahulsingh gmail com linkedi...,0.527871
3,1373684,limese looking detail oriented analytically sk...,2711440,akhil pratap singh asian paints ltd central su...,0.516081
4,1373684,limese looking detail oriented analytically sk...,2500298,piyush kumar patel gmail com mumbai maharashtr...,0.575001
5,1373684,limese looking detail oriented analytically sk...,1229742,work experience projects internships abhishek ...,0.555705
6,1373684,limese looking detail oriented analytically sk...,2681866,vignesh iyer new delhi gmail com linkedin com ...,0.512643
7,1373684,limese looking detail oriented analytically sk...,2436531,amit ahirwar male dob nov gmail com scholastic...,0.615747
8,1373684,limese looking detail oriented analytically sk...,2806722,mithila kirve mumbai maharashtra india mobile ...,0.521031
9,1373684,limese looking detail oriented analytically sk...,218127,rahul boriwal email id gmail com contact profe...,0.551550


In [116]:
m1.loc[m1['job_id']==1373684]

,job_id,description,user_id,resume_text,scores
83,1373684,limese looking detail oriented analytically sk...,2280817,october current may october pranav jaiswal sup...,0.729711
84,1373684,limese looking detail oriented analytically sk...,2730065,prakash murugesan india mobile email gmail com...,0.659762
85,1373684,limese looking detail oriented analytically sk...,2354895,rahul singh email rahulsingh gmail com linkedi...,0.627667
86,1373684,limese looking detail oriented analytically sk...,2711440,akhil pratap singh asian paints ltd central su...,0.626746
87,1373684,limese looking detail oriented analytically sk...,2500298,piyush kumar patel gmail com mumbai maharashtr...,0.615747
88,1373684,limese looking detail oriented analytically sk...,1229742,work experience projects internships abhishek ...,0.607781
89,1373684,limese looking detail oriented analytically sk...,2681866,vignesh iyer new delhi gmail com linkedin com ...,0.575001
90,1373684,limese looking detail oriented analytically sk...,2436531,amit ahirwar male dob nov gmail com scholastic...,0.573481
91,1373684,limese looking detail oriented analytically sk...,2806722,mithila kirve mumbai maharashtra india mobile ...,0.573312
92,1373684,limese looking detail oriented analytically sk...,218127,rahul boriwal email id gmail com contact profe...,0.555705


In [114]:
mdf.loc[mdf['job_id']==1373681]

,job_id,description,user_id,resume_text
0,1373681,company global recognized leader financial tec...,439042,vibha issar email vibhaissar rediffmail com mo...
1,1373681,company global recognized leader financial tec...,2015063,swati kamble gmail com expert financial servic...
2,1373681,company global recognized leader financial tec...,2777927,head office business process review revamping ...
3,1373681,company global recognized leader financial tec...,58130,work experience summary amp key achievements t...
4,1373681,company global recognized leader financial tec...,2290933,work experience yes bank april march stressed ...
5,1373681,company global recognized leader financial tec...,190578,nikhil jain nikhil gmail com experienced profe...
6,1373681,company global recognized leader financial tec...,26047,mobile investments amp corporate strategy stra...
7,1373681,company global recognized leader financial tec...,1128193,vanshika agarwal fixed income research mba gol...
8,1373681,company global recognized leader financial tec...,2800701,g sreenivasa reddy phone email gsnivasareddy g...
9,1373681,company global recognized leader financial tec...,23029,pramod tiwari finance professional amp busines...


In [ ]:
def do_something():
    jobid = main_df

In [51]:
# cl = pd.read_csv('../custom_data/metric_data/query_result_2024-02-16T12 00 06.729814Z.csv')

In [52]:
# cl.determinedPortalNewMl.value_counts()

In [53]:
# cl.determinedPortalNewMl.isnull().sum()

In [55]:
# del cl 